In [16]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
from helpers.rotation import RotateFromTo

from yaml import load
from yaml.loader import Loader


In [17]:
BASENAME = "09_GrotteDeVallorbe"

fp = f"../../real_data/{BASENAME}/params.yaml"

params = load(open(fp), Loader)

GLOBAL_SHIFT = params["alignment"]["globalShift"]

PASSAGE = "Toboggan"

In [18]:
data = pv.read(f"../../real_data/{BASENAME}/output/{PASSAGE}_cleaned.stl").extract_surface().triangulate()

In [19]:
plane_origins = np.loadtxt(f"../../real_data/{BASENAME}/process/planes_as_vectors/{PASSAGE}_z_interp_3d_Planes_Origin.txt") + GLOBAL_SHIFT
plane_vectors =  np.loadtxt(f"../../real_data/{BASENAME}/process/planes_as_vectors/{PASSAGE}_z_interp_3d_Planes.txt")

In [20]:
# define orthogonal unit vectors in horizontal plane 
d0 = np.array([1,0,0])
d1 = np.array([0,1,0])

# project the section-normal vectors onto horizontal plane. 
horiz_dip_dir = np.asarray((plane_vectors.dot(d0), plane_vectors.dot(d1), np.zeros(len(plane_vectors)))).T

In [21]:
print(len(horiz_dip_dir))

292


In [23]:
for n, (orig, vec, vec_horiz) in tqdm(enumerate(zip(plane_origins, plane_vectors, horiz_dip_dir)), total = len(plane_origins)):
    plane = pv.Plane(orig, vec, 20, 20, 100, 100).extract_surface().triangulate()
    inter, _, _ = data.intersection(plane, split_first = False, split_second = False)

    R1 =  RotateFromTo(vec, vec_horiz)
    pp = inter.points - np.mean(plane.points, axis=0)
    pp_vertical = R1 @ pp.T
    R2 = RotateFromTo(vec_horiz, d1)
    pp_rotated = R2 @ pp_vertical

    np.savetxt(f"../../real_data/{BASENAME}/process/sections_as_polylines/{PASSAGE}_unordered_section_{n}.txt", pp_rotated.T)
        
    #pp_reordered = order_points(pp_rotated.T.tolist(), 0)


100%|████████████████████████████████████████████████████████████████████████████████| 292/292 [18:00<00:00,  3.70s/it]
